In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import scipy.optimize as optimize
import sys

from qiskit import Aer, execute, QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.opflow import I, X, Y, Z
from qiskit.opflow import CircuitOp, SummedOp, StateFn
from qiskit.opflow import PauliExpectation, PauliTrotterEvolution, Suzuki
from qiskit.quantum_info import Statevector

cwd = Path(os.getcwd())
sys.path.append(cwd)

from main import *


In [ ]:
def get_heisenberg_int(nb_qubits, j, periodic=True):

    local_terms = []
    for op in [X, Y, Z]:
        for i in range(nb_qubits-1):
            local_terms.append((I^i)^(op^2)^(I^(nb_qubits - i - 2)))
        if periodic:
            local_terms.append(op^(I^(nb_qubits - 2))^op)

    return j*SummedOp(local_terms)

def get_compact_2_qubits_unitary(param):
    """
    Voir https://journals.aps.org/pra/pdf/10.1103/PhysRevA.69.032315
    """
    b = np.pi / 2 - 2 * param
    a = 2 * param - np.pi / 2

    qc = QuantumCircuit(2)
    qc.rz(np.pi/2, 1)
    qc.cx(1, 0)
    qc.rz(a, 0)
    qc.ry(b, 1)
    qc.cx(0, 1)
    qc.ry(a, 1)
    qc.cx(1, 0)
    qc.rz(np.pi / 2, 0)

    return qc
    

def get_ansatz(nb_qubits, depth):
    nb_odd_params_per_layer = nb_qubits // 2  # following naming convention from paper,
                                              # so odd is for entangling between qubits {(0,1),(2,3),...} :| 
    nb_even_params_per_layer = (nb_qubits - 1) // 2
    theta_params = ParameterVector('t', depth * nb_odd_params_per_layer)
    phi_params = ParameterVector('p', depth * nb_even_params_per_layer)

    qc = QuantumCircuit(nb_qubits)
    for l in range(depth):
        for i in range(nb_odd_params_per_layer):
            qc.compose(get_compact_2_qubits_unitary(theta_params[l*nb_odd_params_per_layer + i]), [2*i, 2*i+1], inplace=True)
        for i in range(nb_even_params_per_layer):
            qc.compose(get_compact_2_qubits_unitary(phi_params[l*nb_even_params_per_layer + i]), [2*i+1, 2*(i+1)], inplace=True)

    return qc, theta_params, phi_params

def get_neel_state(nb_qubits):
    qc = QuantumCircuit(nb_qubits)
    qc.x([2 * i + 1 for i in range(nb_qubits // 2)])

    return qc

def state_evolution(state, hamiltonian, tau, num_reps, trotter_order):
    assert (state.num_qubits == hamiltonian.num_qubits)
    t_param = Parameter('t')
    evolution_op = (t_param * hamiltonian).exp_i()
    trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=trotter_order, reps=num_reps)).convert(evolution_op @ CircuitOp(state))
    qc = trotterized_op.bind_parameters({t_param: tau})

    return qc

In [ ]:
nb_qubits = 8
coupling = 1

# Heisenberd Hamiltonian
hamiltonian = get_heisenberg_int(nb_qubits, coupling)

# Initial state to work with
init_state = get_neel_state(nb_qubits)

sv_sim = Aer.get_backend('statevector_simulator')
u_sim = Aer.get_backend('unitary_simulator')

depth_l = 16  # Value used to generate Fig. 5(a)
anzats, t_params, p_params = get_ansatz(nb_qubits, depth_l)

num_params = len(t_params) + len(p_params)

tau = 2.5 / coupling
reps= 16  # Value used to generate Fig. 5(a). Should have: (2 * depth_l + reps) < n_max
          # where n_max is max number of Trotter steps within coherence time of the device
order = 1
qc_evol = state_evolution(state=init_state.compose(anzats), hamiltonian=hamiltonian, tau=tau, num_reps=reps, trotter_order=order)



# Exact diagonalization

$\ket{\psi (t)} = e^{-iHt}\ket{\psi_0}$

The Hamiltonoan $H$ is not diagonal in the computational basis but we can find a transformation $U$ such that $H_W = U^\dagger H_C U$ where $H_C$ is the Hamiltonian matrix in the computational basis and $H_W$ is the Hamiltonian in the basis that diagonalize $H_C$. Hence $H_W$ is diagonal.<br>
The values on the diagonal of $H_W$ are the eigenvalues. The columns of the $U$ matrix are the eigenvectors.<br>
The transformation $U$ allows to transform a vector from the $W$-basis to the $C$-basis and $U^\dagger$ from the $C$-basis to the $W$-basis.

We can therefore write $\ket{\psi (t)} = Ue^{-iH_Wt}U^\dagger \ket{\psi_0}$.

The steps that we have to follow are:

1. Diagonalize $H_C$ to obtain $H_W$ and $U$
2. For a given time $t$, compute $\ket{\psi (t)}$

In [ ]:
# Matrix representation of the Hamiltonian
hamiltonian_matrix = hamiltonian.to_matrix()

# Diagonalization
eigval, eigvec = np.linalg.eigh(hamiltonian_matrix)
u = eigvec
u_dag = np.conj(eigvec).T

# time range for Fig. 5
time = np.arange(0., 1., .01)

# diagonal operators for all t
exp_iHt = np.exp(-1.j * eigval.reshape((-1,1)) * time.reshape((1,-1)))

# state vectors for all t
init_state_sv = execute(init_state, sv_sim).result().get_statevector()
init_state_w_basis = u_dag @ np.asarray(init_state_sv).reshape((-1,1))  # shape (2**nb_qubits, 1), e.g. (256,1)
evol_state_w_basis = exp_iHt * init_state_w_basis  # shape (2**nb_qubits, nb_timesteps), e.g. (256,40)
evol_state_c_basis = u @ evol_state_w_basis  # shape  (2**nb_qubits, nb_timesteps), e.g. (256,40)

In [ ]:
values_x = []
values_y = []
values_z = []
for t in range(len(time)):
    # !!! Pourquoi ça ne fonctionne pas avec PauliExpectation???
    # values_x.append(PauliExpectation().convert(~StateFn(X^nb_qubits)).eval(StateFn(evol_state_c_basis[:,t])))
    values_x.append((evol_state_c_basis[:,t].reshape((1,-1)) @ (X^nb_qubits).to_matrix() @ evol_state_c_basis[:,t].reshape((-1,1))).item())
    values_y.append((evol_state_c_basis[:,t].reshape((1,-1)) @ (Y^nb_qubits).to_matrix() @ evol_state_c_basis[:,t].reshape((-1,1))).item())
    values_z.append((evol_state_c_basis[:,t].reshape((1,-1)) @ (Z^nb_qubits).to_matrix() @ evol_state_c_basis[:,t].reshape((-1,1))).item())
plt.plot(time, values_x, '.', label='$\langle X^{\otimes n}\\rangle $')
plt.plot(time, values_y, '--', label='$\langle Y^{\otimes n}\\rangle $')
plt.plot(time, values_z, label='$\langle Z^{\otimes n}\\rangle $')
plt.legend(ncol=3)

# Optimization

Cost function is $\mathcal{C}(\theta ) = 1 - \mathcal{F}(t, \boldsymbol{\vartheta}^{(l)}) = 1 - \big| \langle \psi (\boldsymbol{\vartheta}^{(l)})|\psi (t) \rangle \big|^2 $

In [ ]:
def get_current_sv(init_state, hamiltonian, current_t):
    hamiltonian_matrix = hamiltonian.to_matrix()
    eigval, eigvec = np.linalg.eigh(hamiltonian_matrix)
    u = eigvec
    u_dag = np.conj(eigvec).T

    sv_sim = Aer.get_backend('statevector_simulator')
    init_state_sv = execute(init_state, sv_sim).result().get_statevector()

    return u @ np.exp(-1.j * eigval.reshape((-1, 1)) * current_t) * u_dag @ np.asarray(init_state_sv).reshape((-1, 1))


def objective_fct(params, init_state, circuit, hamiltonian, current_t):

    grounded_qc = init_state.compose(circuit).bind_parameters(params)
    infidelity = 1 - np.linalg.norm((~StateFn(get_current_sv(init_state, hamiltonian, current_t)) @ StateFn(grounded_qc)).eval()) ** 2

    return infidelity

# Experiment's parameters

In [ ]:
parameters_test = []
def callback(xk):
    global parameters_test
    parameters_test.append(xk)
    

seed = 267
np.random.seed(seed)
init_params = np.random.random(num_params)

epsilon = 1e-4  # optimization threshold for infidelity (1 - F)

current_t = 2.
num_iter = 1
parameters_hist = []
infidelity_hist = []
current_iter = 0
# res = optimize.minimize(fun=objective_fct,
#                         args=(init_state.compose(anzats), hamiltonian, current_t),
#                         x0=init_params,
#                         method='L-BFGS-B',
#                         callback=callback,
#                         jac=None,  # gradient will be estimated using 2-point finite difference estimation with an absolute step size. 
#                         bounds=[(0, 2*np.pi)]*num_params,
#                         options={'maxiter':num_iter,
#                                  'gtol': 1e-05})

In [ ]:
inf_hist = np.load('hist_infidelity_8_267_1000_2.0.npy', allow_pickle=True)
params_hist = np.load('hist_params_8_267_1000_2.0.npy', allow_pickle=True)
params_final = np.load('final_params_8_267_1000_2.0.npy', allow_pickle=True)
plt.plot(inf_hist)

In [ ]:
sv_opt = execute(init_state.compose(anzats).bind_parameters(params_final), sv_sim).result().get_statevector()
np.round(np.asarray(sv_opt), 3)

In [ ]:
objective_fct(params_hist[0,:], init_state, anzats, hamiltonian, current_t)